In [2]:
import requests
import json
import datetime as dt
from hdfs import InsecureClient
client = InsecureClient('http://localhost:9870', user='big') #나의 로컬호스트주소 #user : big(내가정함)

In [3]:
# 라이브러리 입장에서는 404도 응답이기 때문에 에러가 아니다. 
# 우리가 그런상황도 error라고 정해 놓고 가된다.

# raise Exception('응답코드 : ' + str(404))  의도적 에러 Exception: 응답코드 : 404

def execute_rest_api(method, url, headers, params):
    if method == 'get':
        res = requests.get(url,params=params,headers=headers)
    elif method == 'post' :
        res = requests.post(url,params=params,headers=headers)
    
    if res ==None or res.status_code != 200 :
        raise Exception('응답코드 : ' + str(res.status_code))
        
    return res.text

In [4]:
def cal_std_day(befor_day):   
    x = dt.datetime.now() - dt.timedelta(befor_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0'+ str(x.day)  
    return str(year)+ '-' +str(month)+ '-' +str(day)

In [5]:
# 로그만들기
import logging
# 내가원하는 과정중에 발생한 에러만 골라서 기록하는 로깅이 필요
co_logger = logging.getLogger('corona_api')
# 로그를 한파일에만 넣으면 나중에 찾기 힘들고 심지어 열기조차 불가능 할 수 있음 -> 이번엔 날짜별로 하자, 하둡은 파일 크기별로함
handler = logging.FileHandler('./log/rest_api/'+cal_std_day(0)+'.log')
co_logger.addHandler(handler)

In [6]:
from datetime import date, datetime
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pandas as pd
from bs4 import BeautifulSoup

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [7]:
# 시장지표 크롤링
# 인베스팅.com 주요 주가지수

file_dir = '/finance/stock_index/'
file_name = 'stock_index_' + str(cal_std_day(0))+'.json'
url = 'https://comp.fnguide.com/SVO/WooriRenewal/new_overview.asp'
response_txt = execute_rest_api('get',url,{},{})
soup = BeautifulSoup(response_txt, 'html.parser')
trs = soup.find('div',{'id':'right_column'}).find('tbody').findAll('tr')
cols = ['country_name','si_name', 'si_price', 'si_fl_rate', 'si_is_rise']
data = []
for tr in trs :
    rows=[]
    # 국가 이름 : country_name
    rows.append(tr.findAll('td')[0].text)
    # 지수 이름 : index_name
    if tr.findAll('td')[1].text == '종합주가지수':
        rows.append('코스피')
    else :
        rows.append(tr.findAll('td')[1].text)
    # 종가 :price
    rows.append(tr.findAll('td')[2].text.replace(',',''))
    # 등락율 : fluctuation_rate
    rows.append(tr.findAll('td')[3].text.replace('+',''))
    # 상승여부 : is_rise
    if tr.findAll('td')[3].text.replace('%','')[0] == '-' :
        rows.append(False)
        tmp = dict(zip(cols,rows))
        data.append(tmp)
    else :
        rows.append(True)
        tmp = dict(zip(cols,rows))
        data.append(tmp)
res = {
    'meta':{
        'desc':'주요 주가 지수',
        'cols':{
            'country_name':'국가이름','si_name':'주가지수이름', 'si_price':'주가지수종가', 'si_fl_rate':'지수등략률(%)', 'si_is_rise':'지수상승여부'
        },
        'std_day':cal_std_day(0)
    },
    'data':data
}
client.write(file_dir+file_name,json.dumps(res,ensure_ascii=False),encoding='utf-8')

In [11]:
# 채권 크롤링
# 모을 연도 = [1,2,3,5,10,30]
# 모을 국가 한국, 미국, 영국, 일본의 table bond_table_number = [60, 1, 3, 20]
file_dir = '/finance/sovereign_yield/'
file_name = 'sovereign_yield_' + str(cal_std_day(0))+'.json'
cols = ['country_name', '1y','2y','3y','5y','10y','30y']
data = []
for i in range(4):
    tmp = dict(zip(cols,get_bond_yield_row(i)))
    data.append(tmp)
res = {
    'meta':{
        'desc':'한국,미국,영국,일본 국채 금리',
        'cols':{
            'country_name':'국가명','1y':'1년물','2y':'2년물','3y':'3년물','5y':'5년물','10y':'10년물','30y':'30년물'
        },
        'std_day':cal_std_day(0)
    },
    'data':data
}

client.write(file_dir+file_name,json.dumps(res,ensure_ascii=False),encoding='utf-8')

In [9]:
def get_ops(num):
    bond_table_number = ['cr1', 'bond_table_1', 'bond_table_3', 'bond_table_20']
    base_url = 'https://kr.investing.com/rates-bonds/'
    url_list = ['south-korea-government-bonds', 'americas-government-bonds', 'european-government-bonds','asian-pacific-government-bonds']
    file_dir = '/finance/sovereign_yield/'
    file_name = 'sovereign_yield_' + str(cal_std_day(0))+'.json'
    base_url = 'https://kr.investing.com/rates-bonds/'
    url = base_url + url_list[num]
    response_txt = execute_rest_api('get',url,{},{})
    soup = BeautifulSoup(response_txt, 'html.parser')
    return soup.find('table', {'id':bond_table_number[int(num)]})

In [10]:
# 모을 연도 = [1,2,3,5,10,30]
def get_bond_yield_row(num):
    trs = get_ops(num).findAll('tr')[1:]
    bond_rows = []
    if num == 0:
        bond_rows.append("한국")
    elif num == 1:
        bond_rows.append("미국")
    elif num == 2:
        bond_rows.append("영국")
    elif num == 3:
        bond_rows.append("일본")
    else :
        raise
    for tr in trs:
        price_year = tr.find('a').text
        if (('1년' in price_year) or ('2년' in price_year) or ('3년' in price_year) or ('5년' in price_year) \
            or ('10년' in price_year) or ('30년' in price_year)) and ('12년' not in price_year) and ('15년' not in price_year) and ('25년' not in price_year) :
            bond_rows.append(tr.findAll('td')[3].text.replace(',','')) # 종가 채권수익률
    return bond_rows

In [12]:
## 중앙은행의 이자율
file_dir = '/finance/cetral_bank_interest_rate/'
file_name = 'cental_interest_' + str(cal_std_day(0))+'.json'
url = 'https://kr.investing.com/central-banks/'
response_txt = execute_rest_api('get',url,{},{})
soup = BeautifulSoup(response_txt, 'html.parser')
ops = soup.find('table',{'id':'curr_table'}).findAll('tr')[1:]

cols = ['country_name','ctr_bank', 'ctr_inter', 'ctr_next_conf', 'ctr_cng_date', 'ctr_latest_point', 'ctr_is_rise']
data = []
# 중앙은행
cnts = ['한국','미국','유럽','영국','스위스','호주','캐나다','일본','러시아','인도','중국','브라질']
for idx,op in enumerate(ops):
    rows = []
    # 국가이름
    rows.append(cnts[idx])
    # 은행이름
    rows.append(op.findAll('td')[1].text)
    # 현재금리, 다음회의, 최근변동일, 최근변동포인트, 상승여부
    rows.append(op.findAll('td')[2].text.replace('%',''))
    rows.append(op.findAll('td')[3].text.replace('년','').replace('월','').replace('일','').replace(' ','-')[1:][:-1])
    rows.append(op.findAll('td')[4].text[:13].replace('년','').replace('월','').replace('일','').replace(' ','-'))
    rows.append(op.findAll('td')[4].text[14:].replace('(','').replace(')','').replace('bp',''))
    if op.findAll('td')[4].text[14:].replace('(','').replace(')','').replace('bp','')[0] == '-':
        rows.append(False)
        tmp = dict(zip(cols,rows))
        data.append(tmp)
    else:
        rows.append(True)
        tmp = dict(zip(cols,rows))
        data.append(tmp)
res = {
    'meta':{
        'desc':'세계주요국의 중앙은행의 이자율',
        'cols':{
            'country_name':'국가이름','ctr_bank':'중앙은행', 'ctr_inter':'금리(%)', 'ctr_next_conf':'다음 금리결정회의 날짜', 'ctr_cng_date':'최근 금리 결정 날짜',
            'ctr_latest_point':'금리 증감 퍼센트포인트', 'ctr_is_rise':'금리상승여부'
        },
        'std_day':cal_std_day(0)
    },
    'data':data
}
client.write(file_dir+file_name,json.dumps(res,ensure_ascii=False),encoding='utf-8')

In [44]:
int(-1)

-1